# CFD Simulation Case Setup

**The created CFD domain is now read into the CFD package of interest to set the CFD simulation up. It should be noted that the current tutorial has a significant difference compared to other available CFD tutorials online! This tutorial is structured and developed based on a generic and methodological approach to set up a CFD simulation. The first principals and reasonings for each setting is discussed at each step. Potential alterations and modifications to perform similar analysis are also addressed and discussed. Hence, in the end user will have the capability of applying potential modifications, improvements or extending the application of the current CFD simulation to a more complex problem of interest in the end of the tutorial, rather than having a one time successful run of a specific simulation with specific and strictly pre-defined boundary conditions.**

> **_In simple words: Current tutorial teaches users to fish, rather than giving them a fish._**

## Setting up a CFD simulation has following four steps:

1. ###### Setup Model/s:   
According to the physics of the flow field user will select required model/s to simulate the flow.

2. ###### Setup Working Fluid/s & Solid/s:   
User will define the physical and thermodynamical properties of the working fluid/s and solid/s in the problem.    

3. ###### Setup Boundary & Zone Conditions:    
Solving the governing equations of the flow (i.e. system of partial differential equations) requires well-defined boundary conditions within the CFD domain. These conditions are selected and defined in this step.

4. ###### Setup Solution Methods:    
In CFD simulations the governing equations of the flow are solve numerically. Based on the physics of the problem appropriate numerical schemes and solution methods are selected at this step.

In the following section the details for the above four steps for the CFD simulation setup for **3D Particle Dispersion in Turbulent Channel Flow** are explained in great details. It should be noted that the path for defining conditions and other settings are provided in `command line` format. Users can access exact same settings and options by following the provided path via the tree of progress or pull down menu in ANSYS FLUENT. The summary of the steps to take for CFD simulation setup for this problem are as follows:

 1-  `/define/models/steady`   
 2-  `/define/models/solver/pressure-based`  
 3-  `/define/operating-conditions/gravity`   
 4-  `/define/models/viscous/laminar`    
 
 4-  `/define/material/change-create`    
 5-  `/define/boundary-conditions/fluid`   
 6-  `/define/boundary-conditions/velocity-inlet`    
 7-  `/define/boundary-conditions/pressure-outlet`   
 8-  `/define/boundary-conditions/wall`    
 9-  `solve/set/discretization-schem`    
 10- `solve/set/under-relaxation`   
 11- `/solve/initialize/compute-defaults/velocity-inlet`    
 12- `solve/iterate`

Following is the step-by-step explanation for each of the above command/setting procedure:

**1. Setup Model/s:**

**1.0 Setup General Models for Carrier Flow:**

* The current flow field of interest in majority of applications is steady. Meaning that a uniform flow with an almost constant will enter the channel and evolves along it. Therefore, the steady model is chosen: `/define/models/steady`. In case the flow rapidly changes with respect to time the Transient model should be chosen in this step.  

* In majority of industrial applications the speed of the flow in channels is defined in subsonic region. Therefore, variation of density with respect to the pressure can be neglected. As a result of this assumption one can define the working fluid to be incompressible: `/define/models/solver/pressure-based`. In cases that the speed of the flow enters sonic and supersonic regions, the changes in density (i.e. compressibility) of the flow will be an important factor and the solver must be defined as density-based.

* In this problem of interest the gravitational force and it's direction is an important variable to define, since it can have significant effect on particle dispersion. This variable can be set via the path of `/define/operating-conditions/gravity`. The magnitude and direction of it is defined based of the coordinates of the computational domain. In this example the x-component of gravity is set to $-9.8 \frac{m}{s^2}$.    

* In the current problem of interest the flow is viscous and value of it's Reynold number, based on the height of the channel is $1.8$ x $10^5$. Thus, the flow is turbulent. To numerically, by solving the RANS equations, model this flow an appropriate closure model should be selected. It is important to note that the choice of closure model is problem specific. Users are recommended to do an initial search on the capabilities and relevance of different closure models and choose the appropriate one for their specific applications. One of the widely used/suggested closure model to simulate turbulent channel flows is the $k-\omega$ model with $SST$ correction [1]. In FLUENT this model is selected via the path of:`/define/models/viscous/kw-sst`. 

**1.1 Setup Model for Particle Dispersion:**
Inside FLUENT particle dispersion within the flow is simulated via the Discrete Phase Model (DPM). DPM is activated via path of `/define/models/dpm`. Navigation through the setup steps of DPM using command line is possible but rather complex, which will lead into a lengthy tutorial. Hence, users are guided through these setup steps by providing the main command line on topic line and then visualization of DPM's GUI panel as shown in Fig. 1. Readers are referred to the FLUENT User's Guide for more detail on various available options within the DPM.

<img src="./Images/DPM_General.png" width="500">

Fig. 1 - Discrete Phase Model (DPM) activation panel.

**1.1.1 Interaction [`/define/models/dpm/interaction`]**

Users can activate DPM by enabling the `Interaction with Continuous Phase` option. This will enable tracking the dispersion of single particles within the carrier flow using a Lagrangian approach, as discussed in Physics section.  
Here the `Number of Continuous Phase Iterations Per DPM Iteration` is set to 1. This option depends on the problem's physics and availability of computational resources. In the current problem of interest the density ratio of particles to carrier flow $(\frac{\rho_p}{\rho_f})$ is small enough to us 1-way coupling assumption for numerically simulate this flow field. 1-way coupling means that the important physics is the effect of the carrier flow on particles dispersion. Hence, to save computational time and cost users are recommended to obtain the converged solution of the carrier flow first, then using a one time iteration run for continuous phase simulate particle path within the entire computational domain. In case studies that the physics of the flow imposes 2- or 4-ways coupling requirement users should come-up with an appropriate iteration frequency to correctly capture the physics of both the flow and particle dispersion via adequate number of iterations.

**1.1.2 Tracking Tab [`/define/models/dpm/numerics/tracking-parameters`]**

The `Max. Number of Steps` defines the maximum number of time steps that DPM simulates a particle path via integration of force balance on each particle. If the value of this input is too low, the particle path will not be completely modeled. The set number of $5$ x $10^7$ here was chosen based on the length of the channel and a trial and error process. This value would guarantee complete path simulation of all particles within the computational domain.  
The `Specify Length Scale` option is enabled to define the integration time step size of the force balance on particles. This input value is defined according to the modeled particle diameter ($1$ x $10^{-5}$), which is hollow glass in this study.  
The `Drag Parameter` of choice remained untouched to be `spherical` since the assumed shape of particles in the experiment is spherical. Users are recommended to choose the appropriate drag law based on the shape of the particle of interest.

**1.1.3 User Defined Function (UDF) Tab [`/define/models/dpm/user-defined`]**

In FLUENT User Defined Functions (UDFs) are piece of external code written to perform desired computational actions inside the CFD simulations. In this problem of interest to compare the obtained numerical results against theoretical and experimental data, a UDF is written to export simulated particle data. The source file of this UDF is available in Simulation_Src folder on Github. In order to add/active this piece of code users should first put a copy of this file into the working directory. Then use the path `define/user-defined/compiled-functions` to compile the UDF within the solver and in the end choose the compiled UDF from the list of the `Scalar Update` option under `UDF` tab in this panel as shown in Fig. 1. Compile and activation of this UDF would allow user to export obtained particle data after each DPM iteration into a text file inside the working directory. The structure of the exported text file and analysis of it will be provided later in the Post-Processing stage.

**! The rest of the DPM options at this panel will be left as default. Users are recommended to review these inputs based on their problem of interest.**

**1.1.4 Injections `/define/models/dpm/injection`**

After setup the general settings of the DPM, at this stage users set the details of how to inject particles into the carrier flow. This panel is accessible through the `Injections...` bottom of the DPM panel showed in Fig. 1. This panel is called `Set Injection Properties` panel and is shown is Fig. 2:

<img src="./Images/DPM_Injection.png" width="500">

Fig. 2 - The Set Injection Properties panel to define the details of injected particles into the carrier flow.

In the experimental setup [3] particles are "entrained by the air flow through the sampling tube in the side of the expansion and carried to the center of the (wind) tunnel" at the test section [3]. To simulate this condition `Injection Type` is set to `Single`, the `Particle Type` chosen to be `Inert` to define a specific injection point, velocity, material and size for them.  
The material of the particles is defined and chosen to be `hollow_glass`. This material is defined by users based on the experimental condition as discussed in section `2. Setup Working Fluid/s & Solid/s` below. The name of the defined material will appear within the pull-down menu in this panel.  
Once again based on th experimental condition the injection coordinates of particles is set to (0.5588, 0, 0) $[m]$ with the origin to be at the center of computational domain inlet. The velocity of particles are set to (6.55, 0, 0) $[\frac{m}{s}]$ and diameter of 46.5 $[\mu m]$. All these setting within the `Point Properties` tab of the `Set Injection Properties` panel.

The last settings to enable simulation of particle dispersion in turbulent flow is done in the `Turbulent Dispersion` panel of the `Set Injection Properties` panel as shown in Fig. 3. As discussed earlier in the Physics section a `Stochastic Tracking` approach is taken to simulate the particle dispersion within the turbulent carrier flow. To enable this option two options of `Discrete Random Walk (DRW) Model` and `Random Eddy Lifetime` are in the `Turbulent Dispersion` panel as shown in Fig. 3. The `Number of Tries` is set to $500$ that will lead into significant statistical particle data.  
The value of time scale constant, which is the heart of the discussed uncertainty in the DPM-DRW module, by default is $0.15$. At the first try it is recommended to leave this value as the default value. However, it should be pointed out that this value is a strong function of flow regime (i.e. Reynolds number). Hence, a targetive trial and error process is needed to find the appropriate value of this constant according to the characteristics of the turbulent carrier flow. More details on this choice will be provided later in the Post-Processing section.

<img src="./Images/DPM_Injection_DRW.png" width="500">

Fig. 2 - The Turbulent Dispersion Tab inside the Set Injection Properties panel to enable modeling of particle dispersion within a turbulent flow.

**! The rest of the DPM options at this panel will be left as default. Users are recommended to review them based on their problem of interest.**

**2. Setup Working Fluid/s & Solid/s:**  
* The working material is chosen to be Air for this problem `/define/material/change-create` to match the experimental condition [3]. In most of the CFD packages, air is defined is the default working fluid. However, it can be modified through this menu using the pre-defined materials or defining a new material with unique physical and thermodynamical properties.

* The material properties of particle can also be defined using the path `/define/material/change-create` and edit the properties of the default `Inert Particle` materials. According to the experimental condition the density of hollow glass particles are $260~\frac{kg}{m^3}$ and setting this value and changing the name of the material to `hollow_glass` will set the particle material for users and update it in the DPM-DRW model.

**3. Setup Boundary and Zone Conditions:**    
* In this problem the entire CFD domain is filled with the working fluid (i.e. Air). This working fluid is selected form the defined material/s in the previous step:`/define/boundary-conditions/fluid`. Select Air from the available lists of materials.

* The flow enters from the inlet face of the CFD domain with constant velocity of 6.55 [m/s] in x-direction. User sets the inlet face to a velocity-inlet condition by defining the direction and magnitude of the velocity: `/define/boundary-conditions/velocity-inlet`. In cases where the incoming velocity into the CFD domain is not uniform one can define the incoming velocity with the pre-defined directions or generate a User Define Function (UDF) to describe the velocity profile of interest.

* The turbulence settings of the flow at inlet are defined based on `K and Omega` method with values of $0.35~[\frac{m^2}{s^2}]$ and $425~[\frac{1}{s}]$ for the Turbulent Kinetic Energy ($\kappa$) and Specific Dissipation Rate ($\omega$) respectively. These values are set based on the methodology in [1] to match the turbulence decay within the test section in experiment's wind tunnel. More details of this choice will be discussed in Post-Processing section.

* The flow exits the pipe from the outlet face and it's pressure will be equal to atmospheric pressure. `/define/boundary-conditions/pressure-outlet`. It is assumed that gauge pressure at this face is equal to 0. If in the problem of interest, there exist a specific pressure difference between the inlet and outlet, that magnitude can be defined in inlet and the outlet of the pipe.

* The flow is bounded by pipe's walls and interact with them based on the no-slip boundary condition. User assign the no-slip boundary condition to the wall faces of the CFD domain: `/define/boundary-conditions/wall`. If the shear forces and formed boundary layer becomes important in this region user should either provide required mesh resolution to capture the phenomena or set this boundary to free slip condition such that fluid elements would not interact with wall region.

**4. Setup Solution methods:**   
In this step, it is highly recommended to use the default options and settings, unless based on physics of the problem the user is aware of any specific choices. Upon non-smooth convergence and potential divergence of the CFD simulation user can modify and examine various solution methods. To modify the solution methods and controls use the following commands respectively:

`solve/set/discretization-scheme`

`solve/set/under-relaxation`

Now all boundary conditions and settings for the CFD simulation are defined. User can **initialize** the solution through an educated guess to start the iteration process: `/solve/initialize/compute-defaults/velocity-inlet`
Solution initialization would incept the flow field variables, such as velocity and pressure, based on the defined values by user. For the current problem the CFD domain is recommended to be initialize by values of velocity and pressure at the pipe's inlet.

Iteration process for solving the flow field governing equation now shall start till converged solution is obtained:`solve/iterate`. A general rule of thumb for converged solution is to have continuity residuals of 10-3. More details about commenting on validity of solution and convergence criteria will be discussed in next section.

## Source(s)

> [1] Rybalko, E. Loth, D. Lankford, "A lagrangian particle random walk model for hybrid rans/les turbulent flows" Powder Technoly., 221 (2012), pp. 105–113

> [2] Javaherchi, T., August 2014. Numerical investigation of Marine Hydrokinetic Turbines: methodology development for single turbine and small array simulation, and application to flume and full-scale reference models. Ph.D. Dissertation, University of Washington.

> [3] W.H. Snyder, J.L. Lumley, "Some measurments of particle velocity autocorelation function in turbulent flow." J. Fluid Mech., 48 (1971), pp. 41–71
